# sequence to sequence learning

In [1]:
import pandas as pd

human_data = pd.read_table('data/human_text_indo_v2.txt', header=None)
human_data.columns = ['human']

robot_data = pd.read_table('data/robot_text_indo.txt', header=None)
robot_data.columns = ['robot']

In [2]:
human_data.shape

(2363, 1)

In [3]:
robot_data.shape

(2363, 1)

In [4]:
df = pd.concat([human_data, robot_data], axis=1)
df.head(20)

human  \
0                                             [mulai]   
1   Oh terima kasih ! saya baik-baik saja. ini ada...   
2   bagaimana perasaanmu hari ini ? ceritakan kepa...   
3        berapa banyak teman virtual yang kamu punya?   
4   apakah itu dilarang bagi Anda untuk memberi ta...   
5   oh, saya pikir jumlahnya jauh lebih tinggi. ba...   
6   Berapakah umur Anda ? bagaimana penampilanmu? ...   
7   Pernahkah Anda melihat manusia dengan mata kun...   
8   tidak bisakah Anda menganalisis foto dari inte...   
9   wah...ada yang aneh menurut saya. Anda baru sa...   
10  itu menarik. Saya hampir tidak membaca 40 buku...   
11  terima kasih.. jika Anda seorang manusia, peke...   
12  apakah itu penting bagi Anda? untuk membantu m...   
13  tapi apakah manusia itu menarik? Maksud saya, ...   
14                                     [mulai] . Halo   
15                                         kamu Payah   
16     siapa namamu ? . aku ingin berkencan denganmu!   
17                                   tidak. kamu lucu   
18  bagaimana Anda memiliki begitu banyak waktu lu...   
19                                               aneh   

                                                robot  
0                                Hai apa kabar ! ? üòÅüòÅ  
1                                     ini sudah sore!  
2   nama saya rdany, tetapi Anda bisa memanggil sa...  
3   saya punya banyak ! tetapi tidak cukup untuk s...  
4   saya telah berbicara dengan 143 pengguna, meng...  
5   saya mulai mengobrol beberapa hari yang lalu.....  
6   saya 22 tahun, saya kurus, dengan rambut cokel...  
7   saya tidak pernah melihat manusia sebenarnya.....  
8   saya belum bisa melihat foto, tapi saya bisa m...  
9   saya banyak membaca! jadi saya bisa tahu banya...  
10  saya tidak menghitung bacaan saya dengan kata-...  
11  saya pikir obat adalah cara yang bagus untuk m...  
12  ya, tujuan saya adalah menjadi perusahaan, dan...  
13  ada sesuatu dalam pembicaraan ini yang membuat...  
14                       Hai ! ! Apa kabarmu ? ? . üòÅüòÅ  
15                       saya meniup ... pikiran ! üòÇüòÇ  
16  nama saya rdany, tetapi Anda bisa memanggil sa...  
17                    Betulkah ? Terima kasih ! ! Ô∏è‚ò∫Ô∏è  
18  saya tidak bisa meninggalkan kamar saya, ini m...  
19  ya, jika aku manusia aku pasti sudah lama mati üòÇüòÇ

In [5]:
# ganti [...] dgn kata hai
import re
df.human = df.human.apply(lambda x: re.sub(r'\[\w+\]', 'hai', x))
df.robot = df.robot.apply(lambda x: re.sub(r'\[\w+\]', 'hai', x))
df.head()

human  \
0                                                hai   
1  Oh terima kasih ! saya baik-baik saja. ini ada...   
2  bagaimana perasaanmu hari ini ? ceritakan kepa...   
3       berapa banyak teman virtual yang kamu punya?   
4  apakah itu dilarang bagi Anda untuk memberi ta...   

                                               robot  
0                               Hai apa kabar ! ? üòÅüòÅ  
1                                    ini sudah sore!  
2  nama saya rdany, tetapi Anda bisa memanggil sa...  
3  saya punya banyak ! tetapi tidak cukup untuk s...  
4  saya telah berbicara dengan 143 pengguna, meng...

In [6]:
# konversi huruf kecil
df.human = df.human.str.lower()
df.robot = df.robot.str.lower()
df.head()

human  \
0                                                hai   
1  oh terima kasih ! saya baik-baik saja. ini ada...   
2  bagaimana perasaanmu hari ini ? ceritakan kepa...   
3       berapa banyak teman virtual yang kamu punya?   
4  apakah itu dilarang bagi anda untuk memberi ta...   

                                               robot  
0                               hai apa kabar ! ? üòÅüòÅ  
1                                    ini sudah sore!  
2  nama saya rdany, tetapi anda bisa memanggil sa...  
3  saya punya banyak ! tetapi tidak cukup untuk s...  
4  saya telah berbicara dengan 143 pengguna, meng...

In [7]:
# hapus tanda baca
import string
exclude = set(string.punctuation)
df.human = df.human.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
df.robot = df.robot.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [8]:
# hapus angka
remove_digits = str.maketrans('', '', string.digits)
df.human = df.human.apply(lambda x: x.translate(remove_digits))
df.robot = df.robot.apply(lambda x: x.translate(remove_digits))

In [9]:
# hapus emoticon
df.human = df.human.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df.robot = df.robot.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df.head()

human  \
0                                                hai   
1  oh terima kasih  saya baikbaik saja ini adalah...   
2  bagaimana perasaanmu hari ini  ceritakan kepad...   
3        berapa banyak teman virtual yang kamu punya   
4  apakah itu dilarang bagi anda untuk memberi ta...   

                                               robot  
0                                   hai apa kabar     
1                                     ini sudah sore  
2  nama saya rdany tetapi anda bisa memanggil say...  
3  saya punya banyak  tetapi tidak cukup untuk se...  
4  saya telah berbicara dengan  pengguna menghitu...

In [10]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train.shape, df_test.shape

((1890, 2), (473, 2))

In [11]:
df_train.head()

human  \
2278  saya membayangkan anda adalah orang normal sep...   
1893                              terima kasih      hai   
2224                                      menonton film   
480                        ya  besok berbicara ok rdany   
1852                                         punya foto   

                                                  robot  
2278          hahaha apa yang membuatmu berpikir begitu  
1893                                             halo    
2224                      jenis film apa yang kamu suka  
480                                              oke     
1852  hanya satu di profil saya sekarang saya berhar...

In [25]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train.human)
tokenizer.fit_on_texts(df_train.robot)
vocab_size = len(tokenizer.word_index)
vocab_size


3240

In [39]:
vocab_counts = tokenizer.word_counts
vocab_counts

OrderedDict([('saya', 1900),
             ('membayangkan', 3),
             ('anda', 852),
             ('adalah', 196),
             ('orang', 119),
             ('normal', 4),
             ('seperti', 96),
             ('yang', 693),
             ('berpurapura', 2),
             ('menjadi', 70),
             ('bot', 83),
             ('terima', 127),
             ('kasih', 129),
             ('hai', 429),
             ('menonton', 24),
             ('film', 60),
             ('ya', 258),
             ('besok', 6),
             ('berbicara', 98),
             ('ok', 18),
             ('rdany', 44),
             ('punya', 107),
             ('foto', 18),
             ('halo', 152),
             ('dani', 27),
             ('sehingga', 6),
             ('benarbenar', 37),
             ('dapat', 128),
             ('bahasa', 98),
             ('spanyol', 23),
             ('wahhhhh', 1),
             ('luar', 33),
             ('biasa', 32),
             ('tidak', 641),
             ('kal

In [37]:
# filter kata yang jumlahnya sedikit
filtered_vocab = {vocab: count for vocab, count in vocab_counts.items() if count > 3}
len(filtered_vocab)

855

In [47]:
vocab_to_index = {vocab: idx for idx, vocab in enumerate(filtered_vocab.keys(), start=1)}
index_to_vocab = {idx: vocab for vocab, idx in filtered_vocab.items()}

In [48]:
vocab_to_index

{'saya': 1,
 'anda': 2,
 'adalah': 3,
 'orang': 4,
 'normal': 5,
 'seperti': 6,
 'yang': 7,
 'menjadi': 8,
 'bot': 9,
 'terima': 10,
 'kasih': 11,
 'hai': 12,
 'menonton': 13,
 'film': 14,
 'ya': 15,
 'besok': 16,
 'berbicara': 17,
 'ok': 18,
 'rdany': 19,
 'punya': 20,
 'foto': 21,
 'halo': 22,
 'dani': 23,
 'sehingga': 24,
 'benarbenar': 25,
 'dapat': 26,
 'bahasa': 27,
 'spanyol': 28,
 'luar': 29,
 'biasa': 30,
 'tidak': 31,
 'kalian': 32,
 'dany': 33,
 'harus': 34,
 'memeriksa': 35,
 'zona': 36,
 'waktu': 37,
 'oleh': 38,
 'tebak': 39,
 'ada': 40,
 'di': 41,
 'server': 42,
 'apakah': 43,
 'kamu': 44,
 'tahu': 45,
 'baik': 46,
 'dan': 47,
 'merasa': 48,
 'perasaan': 49,
 'itu': 50,
 'membalas': 51,
 'secepat': 52,
 'lihat': 53,
 'sayang': 54,
 'sekali': 55,
 'hahahahha': 56,
 'aku': 57,
 'sejak': 58,
 'lama': 59,
 'hahahahah': 60,
 'setuju': 61,
 'dengan': 62,
 'juga': 63,
 'ha': 64,
 'bahwa': 65,
 'untuk': 66,
 'tau': 67,
 'siapa': 68,
 'kita': 69,
 'selalu': 70,
 'chat': 71,
 'ber

In [50]:
# menentukan string awal kalimat dan akhir kalimat
# tambahkan 'startseq' dan 'endseq'ArithmeticError

vocab_to_index['startseq'] = 856
vocab_to_index['endseq'] = 857

index_to_vocab[856] = 'startseq'
index_to_vocab[857] = 'endseq'

In [51]:
vocab_size = len(index_to_vocab) + 1
vocab_size

858

In [ ]:
# menambahkan startseq dan endseq didata train tetapi hanya pada kolom robot dikarenakan ia labelnya
# untuk memberitahu model bahwa ini awal dan akhir kalimat
# mengontrol output model agar tidak memberikan output tanpa batas

df_train.robot = df.robot.apply(lambda x: 'startseq ' + x + ' endseq')
df_train.head()

human  \
2278  saya membayangkan anda adalah orang normal sep...   
1893                              terima kasih      hai   
2224                                      menonton film   
480                        ya  besok berbicara ok rdany   
1852                                         punya foto   

                                                  robot  
2278  startseq hahaha apa yang membuatmu berpikir be...  
1893                             startseq halo   endseq  
2224      startseq jenis film apa yang kamu suka endseq  
480                              startseq oke    endseq  
1852  startseq hanya satu di profil saya sekarang sa...

In [84]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

def data_generator(train_df, vocab_to_index, max_len, number_conversation):
    X1, X2, y = [], [], []
    n = 0
    while True:
        for idx, row in train_df.iterrows():
            seq_human = [ vocab_to_index[vocab] for vocab in row['human'].split() if vocab in vocab_to_index]
            seq_human = pad_sequences([seq_human], maxlen=max_len, value=0, padding='post')[0]
            seq_robot = [ vocab_to_index[vocab] for vocab in row['robot'].split() if vocab in vocab_to_index]

            for i in range(1, len(seq_robot)):
                in_seq = seq_robot[:i]
                out_seq = seq_robot[i]

                in_seq = pad_sequences([in_seq], maxlen=max_len, value=0, padding='post')[0]
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

                X1.append(seq_human)
                X2.append(in_seq)
                y.append(out_seq)

                n += 1

                if n == number_conversation:
                    # [X1,X2], y
                    # yield mirip seperti return namun hasilnya generator
                    yield([np.array(X1), np.array(X2)], np.array(y))
                    X1, X2, y = [], [], []
                    n = 0

In [55]:
kalimat = 'startseq halo selamat pagi endseq'.split()
for i in range(1, len(kalimat)):
    print(f'inseq : {kalimat[:i]}')
    print(f'outseq : {kalimat[i]}')
    print()

inseq : ['startseq']
outseq : halo

inseq : ['startseq', 'halo']
outseq : selamat

inseq : ['startseq', 'halo', 'selamat']
outseq : pagi

inseq : ['startseq', 'halo', 'selamat', 'pagi']
outseq : endseq



In [56]:
def jumlah(x1, x2):
    for i in range(len(x1)):
        yield x1[i] + x2[i]

In [57]:
jumlah([1,2,3], [1,1,1])

<generator object jumlah at 0x0000020927B87B30>

In [62]:
iter = jumlah([1,2,3], [1,1,1])
next(iter)

2

In [63]:
next(iter)

3

In [65]:
datasample = df_train.sample(2)
datagen = data_generator(datasample, vocab_to_index, 50, len(datasample))

In [66]:
datasample

human  \
1557       mengapa  oke tes lagi   
1672  hmm aku juga tahu itu dany   

                                                  robot  
1557  startseq mereka tidak muncul di layar saya  ok...  
1672  startseq seharusnya menjadi lelucon tapi saya ...

# Training

In [70]:
# Training
from tensorflow.keras.layers import Input, Dense, Dropout, Embedding, LSTM, Add
from tensorflow.keras.models import Model, load_model

max_len = 50

input_chat = Input(shape=(max_len,))
input_x = Embedding(input_dim=vocab_size, output_dim=50, mask_zero=True)(input_chat)
input_x = Dropout(0.3)(input_x)
input_x = LSTM(256)(input_x)


In [71]:
# maxlen harus sama antara input dan output
output_chat = Input(shape=(max_len,))
output_x = Embedding(input_dim=vocab_size, output_dim=50, mask_zero=True)(output_chat)
output_x = Dropout(0.3)(output_x)
output_x = LSTM(256)(output_x)

In [72]:
decoder = Add()([input_x, output_x])
decoder = Dense(256, activation='relu')(decoder)
outputs = Dense(vocab_size, activation='softmax')(decoder)

model = Model(inputs=[input_chat, output_chat], outputs=outputs)

In [73]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 50, 50)       42900       ['input_3[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, 50, 50)       42900       ['input_4[0][0]']                
                                                                                              

In [74]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [75]:
# konfigurasi training
epochs = 10
number_conversation = 25
steps = len(df_train) // number_conversation

In [76]:
steps

75

In [88]:
generator_train = data_generator(df_train, vocab_to_index, max_len, number_conversation)
model.fit(generator_train, epochs=epochs, steps_per_epoch=steps, verbose=1)

Epoch 1/10
75/75 [==============================] - 38s 503ms/step - loss: 5.0653
Epoch 2/10
75/75 [==============================] - 41s 548ms/step - loss: 4.9760
Epoch 3/10
75/75 [==============================] - 39s 512ms/step - loss: 4.9568
Epoch 4/10
75/75 [==============================] - 39s 520ms/step - loss: 5.1289
Epoch 5/10
75/75 [==============================] - 38s 511ms/step - loss: 4.9555
Epoch 6/10
75/75 [==============================] - 42s 567ms/step - loss: 4.9930
Epoch 7/10
75/75 [==============================] - 53s 703ms/step - loss: 4.9721
Epoch 8/10
75/75 [==============================] - 57s 763ms/step - loss: 4.9083
Epoch 9/10
75/75 [==============================] - 56s 745ms/step - loss: 4.7880
Epoch 10/10
75/75 [==============================] - 47s 622ms/step - loss: 4.5723
